# Phase 1 Review

Phase 1 in Dyalog APL Competition is mainly to let the participants explore the various built-ins of APL (sometimes Dyalog APL specific ones). Following it, this review is also focused on possible combinations of built-ins that can solve each problem.

## Problem 1: Let's Split!

Write a function that, given a right argument `Y` which is a scalar or a non-empty vector and a left argument `X` which is a single non-zero integer so that its absolute value is less or equal to `≢Y`, splits `Y` into a vector of two vectors according to `X`, as follows:

* If `X>0`, the first vector contains the first `X` elements of `Y` and the second vector contains the remaining elements.
* If `X<0`, the second vector contains the last `|X` elements of `Y` and the first vector contains the remaining elements.

### Finding a solution

The suggested built-in is Take `⍺↑⍵`, which is the obvious choice because:

* If `⍺≥0`, `⍺↑⍵` takes first `⍺` cells of `⍵`, and
* If `⍺<0`, `⍺↑⍵` takes last `-⍺` cells of `⍵`.

Pair this with Drop `⍺↓⍵`, which exactly does the above except it "drops" instead of "takes", and we get the exact two chunks we need as given in the description.

In [1]:
f←{(⍺↑⍵)(⍺↓⍵)}
(1 f ⍳5)(5 f ⍳5)(¯3 f ⍳5)(¯5 f ⍳5)

Rebuilding user command cache... done

┌───────────┬────────────┬───────────┬────────────┐
│┌─┬───────┐│┌─────────┬┐│┌─────┬───┐│┌─────────┬┐│
││1│2 3 4 5│││1 2 3 4 5││││3 4 5│1 2│││1 2 3 4 5│││
│└─┴───────┘│└─────────┴┘│└─────┴───┘│└─────────┴┘│
└───────────┴────────────┴───────────┴────────────┘

The only problem is that we don't get the correct order of chunks when `⍺` is negative; the outputs should be `(1 2)(3 4 5)` and `⍬(1 2 3 4 5)` respectively. We can fix that in multiple ways:

* Use a syntax: Use a dfn guard `{cond:expr1 ⋄ expr2}` to do a case analysis based on the sign of `⍺`.
* Solve mathematically: Convert the negative `⍺` to the corresponding positive `⍺`.
* Use the Reverse `⌽⍵` or Rotate `⍺⌽⍵` to reorder the cells.
    * To run Reverse conditionally, we need Power Operator `f⍣x` so that it can decide if it should run (1 time) or not (0 times).
    * Rotation by 1 `1⌽⍵` is the same as reversal for a 2-item vector.

In [2]:
f1←{⍺>0:(⍺↑⍵)(⍺↓⍵) ⋄ (⍺↓⍵)(⍺↑⍵)}  ⍝ Using dfn guard
f2←{(⍺+(≢⍵)×⍺<0){(⍺↑⍵)(⍺↓⍵)}⍵}    ⍝ Converting negative ⍺ to positive
f3←{⌽⍣(⍺<0)⊢(⍺↑⍵)(⍺↓⍵)}           ⍝ Using ⍣
f4←{(⍺<0)⌽(⍺↑⍵)(⍺↓⍵)}             ⍝ Using ⍺⌽⍵
(1 f1 ⍳5)(5 f1 ⍳5)(¯3 f1 ⍳5)(¯5 f1 ⍳5)
(1 f2 ⍳5)(5 f2 ⍳5)(¯3 f2 ⍳5)(¯5 f2 ⍳5)
(1 f3 ⍳5)(5 f3 ⍳5)(¯3 f3 ⍳5)(¯5 f3 ⍳5)
(1 f4 ⍳5)(5 f4 ⍳5)(¯3 f4 ⍳5)(¯5 f4 ⍳5)

┌───────────┬────────────┬───────────┬────────────┐
│┌─┬───────┐│┌─────────┬┐│┌───┬─────┐│┌┬─────────┐│
││1│2 3 4 5│││1 2 3 4 5││││1 2│3 4 5││││1 2 3 4 5││
│└─┴───────┘│└─────────┴┘│└───┴─────┘│└┴─────────┘│
└───────────┴────────────┴───────────┴────────────┘

┌───────────┬────────────┬───────────┬────────────┐
│┌─┬───────┐│┌─────────┬┐│┌───┬─────┐│┌┬─────────┐│
││1│2 3 4 5│││1 2 3 4 5││││1 2│3 4 5││││1 2 3 4 5││
│└─┴───────┘│└─────────┴┘│└───┴─────┘│└┴─────────┘│
└───────────┴────────────┴───────────┴────────────┘

┌───────────┬────────────┬───────────┬────────────┐
│┌─┬───────┐│┌─────────┬┐│┌───┬─────┐│┌┬─────────┐│
││1│2 3 4 5│││1 2 3 4 5││││1 2│3 4 5││││1 2 3 4 5││
│└─┴───────┘│└─────────┴┘│└───┴─────┘│└┴─────────┘│
└───────────┴────────────┴───────────┴────────────┘

┌───────────┬────────────┬───────────┬────────────┐
│┌─┬───────┐│┌─────────┬┐│┌───┬─────┐│┌┬─────────┐│
││1│2 3 4 5│││1 2 3 4 5││││1 2│3 4 5││││1 2 3 4 5││
│└─┴───────┘│└─────────┴┘│└───┴─────┘│└┴─────────┘│
└───────────┴────────────┴───────────┴────────────┘

### Other possible alternatives

Since Dyalog APL 18.0, we get an extension to Partitioned Enclose `⍺⊂⍵` and the inverse of Indices `⍸⍣¯1⊢⍵`, whose combination can give an interesting take (pun intended) to the task. We want two chunks of `⍵`, one always starting at the first item and the other starting at an adjusted value of `⍺`:

In [3]:
(1 1⊂⍳5)(1 0 0 0 0 1⊂⍳5)((,2)⊂⍳5)

┌───────────┬────────────┬────────────┐
│┌─┬───────┐│┌─────────┬┐│┌┬─────────┐│
││1│2 3 4 5│││1 2 3 4 5│││││1 2 3 4 5││
│└─┴───────┘│└─────────┴┘│└┴─────────┘│
└───────────┴────────────┴────────────┘

The left arguments `1 1` or `1 0 0 0 0 1` can be generated from `⍸⍣¯1` (I use `⎕IO←0` to show you how the `⍺`s map to the second element):

In [4]:
{⎕IO←0 ⋄ ⍸⍣¯1⊢⍵}¨ (0 1)(0 5)(0 0)

┌───┬───────────┬─┐
│1 1│1 0 0 0 0 1│2│
└───┴───────────┴─┘

We can combine the two steps and use the mathematical approach from the previous section:

In [5]:
f5←{⎕IO←0 ⋄ (⍸⍣¯1⊢0,⍺+(≢⍵)×⍺<0)⊂⍵}
(1 f5 ⍳5)(5 f5 ⍳5)(¯3 f5 ⍳5)(¯5 f5 ⍳5)

┌───────────┬────────────┬───────────┬────────────┐
│┌─┬───────┐│┌─────────┬┐│┌───┬─────┐│┌┬─────────┐│
││1│2 3 4 5│││1 2 3 4 5││││1 2│3 4 5││││1 2 3 4 5││
│└─┴───────┘│└─────────┴┘│└───┴─────┘│└┴─────────┘│
└───────────┴────────────┴───────────┴────────────┘